# Tutorial 8 - Solver options

In [Tutorial 7](./tutorial-7-model-options.ipynb) we saw how to change the model options. In this tutorial we will show how to pass options to the solver.

All models in PyBaMM have a default solver which is typically different depending on whether the model results in a system of ordinary differential equations (ODEs) or differential algebraic equations (DAEs). 

One of the most common options you will want to change is the solver tolerances. By default all tolerances are set to $10^{-6}$. However, depending on your simulation you may find you want to tighten the tolerances to obtain a more accurate solution, or you may want to loosen the tolerances to reduce the solve time. It is good practice to conduct a tolerance study, where you simulate the same problem with a tighter tolerances and compare the results. We do not show how to do this here, but we give an example of a mesh resolution study in the [next tutorial](./tutorial-9-changing-the-mesh.ipynb), which is conducted in a similar way.

In [1]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm

Note: you may need to restart the kernel to use updated packages.


Here we will change the absolute and relative tolerances, as well as the "mode" of the `CasadiSolver`. For a list of all the solver options  please consult the [documentation](https://docs.pybamm.org/en/latest/source/api/solvers/index.html).

The `CasadiSolver` can operate in a number of modes, including "safe" (default) and "fast". Safe mode performs step-and-check integration and supports event handling (e.g. you can integrate until you hit a certain voltage), and is the recommended for simulations of a full charge or discharge. Fast mode performs direct integration, ignoring events, and is recommended when simulating a drive cycle or other simulation where no events should be triggered.

We'll solve the DFN with all the default options in both "safe" and "fast" mode and compare the solutions. For both simulations we'll use $10^{-3}$ for both the absolute and relative tolerance. For demonstration purposes we'll change the cut-off voltage to 3.6V so we can observe the different behaviour of the two solver modes.

In [2]:
# load model and parameters
model = pybamm.lithium_ion.DFN()
param = model.default_parameter_values
param["Lower voltage cut-off [V]"] = 3.6

# load solvers
safe_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="safe")
fast_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="fast")

# create simulations
safe_sim = pybamm.Simulation(model, parameter_values=param, solver=safe_solver)
fast_sim = pybamm.Simulation(model, parameter_values=param, solver=fast_solver)

# solve
safe_sim.solve([0, 3600])
print(f"Safe mode solve time: {safe_sim.solution.solve_time}")
fast_sim.solve([0, 3600])
print(f"Fast mode solve time: {fast_sim.solution.solve_time}")

# plot solutions
pybamm.dynamic_plot([safe_sim, fast_sim])

Safe mode solve time: 297.861 ms
Fast mode solve time: 100.307 ms


interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

We see that both solvers give the same solution up to the time at which the cut-off voltage is reached. At this point the solver using "safe" mode stops, but the solver using "fast" mode carries on integrating until the final time. As its name suggests, "fast" mode integrates more quickly that "safe" mode, but is unsuitable if your simulation required events to be handled.

Usually the default solver options provide a good combination of speed and accuracy, but we encourage you to investigate different solvers and options to find the best combination for your problem.

In the [next tutorial](./tutorial-9-changing-the-mesh.ipynb) we show how to change the mesh.

## References

The relevant papers for this notebook are:

In [3]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[3] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[4] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with electrolyte. Journal of The Electrochemical Society, 166(15):A3693–A3706, 2019. doi:10.1149/